In [1]:
from __future__ import division, print_function, absolute_import
import sys
import argparse
import json
import os
import time
import warnings

import cv2
import numpy as np
import torch
from PIL import Image
from torchvision import transforms

from deep_sort import nn_matching
from deep_sort import preprocessing
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from mgn.network import MGN
# from coordinate_transform import CoordTrans
from mgn.utils.extract_feature import extract_feature
from yolo import YOLO
from utils.util import checkPoint


Using TensorFlow backend.
/home/lyz/miniconda3/envs/py36gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lyz/miniconda3/envs/py36gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lyz/miniconda3/envs/py36gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lyz/miniconda3/en

/home/lyz/miniconda3/envs/py36gpu/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lyz/miniconda3/envs/py36gpu/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lyz/miniconda3/envs/py36gpu/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lyz/miniconda3/envs/py

In [23]:
def load_model_pytorch(model_path):
    model = MGN()
    model = model.to('cuda')
    model.load_state_dict(torch.load(model_path))
    return model

#保存检测过程中的bbox
def box_encode(model, img, boxes=None, prefix='./img1/'):
    if boxes == None:
        boxes = []
        boxes.append([0,0, img.size[0], img.size[1]])
    imgs = []
#     print(img.size)
    for box in boxes:
        _box = box.copy()
        _box[2] = _box[0] + _box[2]
        _box[3] = _box[1] + _box[3]
        imgs.append(img.crop(_box))
#         print(img.crop(_box).size)

    # img.save(os.path.join(prefix, 'raw.jpg'), quality=95)
    # for idx, x in enumerate(imgs):
    #     x.save(os.path.join(prefix, 'box'+str(idx)+'.jpg'), quality=95)

    model.eval()
    features = torch.FloatTensor()
    test_transform = transforms.Compose([
        transforms.Resize((384, 128), interpolation=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    for _img in imgs:
        _img = test_transform(_img)
        query_feature = extract_feature(model, [(torch.unsqueeze(_img, 0), 1)])
        features = torch.cat((features, query_feature), 0)
    return features.numpy()
def test(img1, img2, _model):
    img1 = Image.fromarray(img1[...,::-1])
    img2 = Image.fromarray(img2[...,::-1])
    fea1 = box_encode(_model, img1)
    fea2 = box_encode(_model, img2)
    dis = np.dot(fea1[0], fea2[0])
    
    return dis

In [6]:
_model = load_model_pytorch('/home/lyz/Desktop/ReID-MGN/model.pt')

In [25]:
img_path = '/home/lyz/Desktop/ICBC/manager/'
# print(list(os.listdir(img_path)))
ans = []
for i in os.listdir(img_path):
    
    if i.split('.')[-1] != 'png':continue
    
    for j in os.listdir(img_path):
        if j.split('.')[-1] != 'png':continue
        
        img1 = cv2.imread(img_path+i)
        img2 = cv2.imread(img_path+j)
        dis = test(img1, img2, _model)
        ans.append(dis)
#         print(test(img1, img2, _model))

In [26]:
print(np.mean(ans))

0.85996115
